# How to authenticate this server for accessing Earth Engine

## Step 1 - Click on the following link
[Link to generate an authentication code for accessing Earth Engine](https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fwww.googleapi
s.com%2Fauth%2Fearthengine.readonly&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&clie
nt_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com)
  
## Step 2 - Click on Accept

## Step 3 - Copy the authentication code that is returned

## Step 4 - Paste the authentication code below

In [ ]:
auth_code ='PASTE_AUTHENTICATION_CODE_HERE'

## Step 5 - From the Cell menu, select "Run All"

In [ ]:
import errno
import json
import os
import urllib
import urllib2

from ee.oauthinfo import OAuthInfo

# This URI prompts user to copy and paste a code after successful
# authorization.
redirect_uri = 'urn:ietf:wg:oauth:2.0:oob'

token_request_params = {
  'code': auth_code,
  'client_id': OAuthInfo.CLIENT_ID,
  'client_secret': OAuthInfo.CLIENT_SECRET,
  'redirect_uri': redirect_uri,
  'grant_type': 'authorization_code'
}

refresh_token = None
try:
  response = urllib2.urlopen('https://accounts.google.com/o/oauth2/token',
                           urllib.urlencode(token_request_params)).read()
  tokens = json.loads(response)
  refresh_token = tokens['refresh_token']
except urllib2.HTTPError, e:
  raise Exception('Problem requesting tokens.  Please try again.  %s %s' %
                (e, e.read()))

### Write refresh token to filesystem for later use
credentials_path = OAuthInfo.credentials_path()
dirname = os.path.dirname(credentials_path)
try:
  os.makedirs(dirname)
except OSError, e:
  if e.errno != errno.EEXIST:
    raise Exception('Error creating %s: %s' % (dirname, e))

  json.dump({'refresh_token': refresh_token}, open(credentials_path, 'w'))

  print '\nSuccessfully saved authorization to %s' % credentials_path

## Step 6 - Verify successful authentication to Earth Engine

In [ ]:
import ee
try:
  ee.Initialize()
  print '\nSuccessfully authenticated to Earth Engine!'
except e:
  print '\nOops. Something went wrong!'
